In [2]:
import rasterio
from rasterio.plot import show
from rasterio.windows import from_bounds
from rasterio.features import rasterize

import pandas as pd
import geopandas as geopd

import numpy as np
import numpy.ma as ma

from tqdm.notebook import tqdm

import os
import warnings

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
plt.rcParams["figure.dpi"]  = 400
plt.style.use('dark_background')

In [3]:
# Paths to data sources
src_path = "/path/to/filename.gpkg"


land_cover_paths = {2000: "/path/to/filename.tif", 
    2006: "/path/to/filename.tif",
    2012: "/path/to/filename.tif",
    2018: "/path/to/filename.tif"
}

lc_classes = {1 : "crop", 2 : "grass", 3 : "shrubs",
              4 : "dwood", 5: "ewood", 6: "urban",
              7:"inwater", 8: "bares", 9: "wetland"
             } # Check that your data source has the same classification
# Getting all the land cover column names
lc_columns = []
for lc_year in land_cover_paths:
    lc_columns.extend([f"{lc_classes[key]}_perc_{lc_year}" for key in lc_classes.keys()])

soil_path = "/path/to/filename.gpkg"
soil_classes = {
    "bedrock": [195110, 195111, 195112, 195312],
    "glaciofluvial": [195310],
    "silt": [195410],
    "till": [195210],
    "clay": [195413, 195511, 195618],
    "peat": [19551891, 19551892, 19551822]
    } # Check that your data source has the same classification
soil_columns = [f"{key}_perc" for key in soil_classes]

soil_depth_path = "/path/to/filename.tif"

slope_path = "/path/to/filename.tif"

dem_path = "/path/to/filename.vrt"

In [4]:
# Load watersheds data
watersheds = geopd.read_file(src_path, layer='attributes_v1')
# Deleting sliver polygons
watersheds = watersheds[watersheds.area > 100]
watersheds = watersheds.reset_index(drop=True)

In [ ]:
watersheds

In [6]:
def add_lc_to_watersheds(subwatersheds, land_cover_path, lc_classes, year):
    """
    Adds land cover classification proportions of a spesific year to subwatersheds.

    Parameters:
    subwatersheds (GeoDataFrame): A GeoDataFrame containing subwatershed geometries and attributes.
    land_cover_path (str): Path to the land cover raster file.
    lc_classes (dict): A dictionary mapping land cover class values to their names.
    year (int): The year of the land cover data.
    bounds (tuple): A tuple containing the bounding box coordinates (minx, miny, maxx, maxy).

    Returns:
    GeoDataFrame: The input GeoDataFrame with additional columns for each land cover class proportion.
    """
    place_id = subwatersheds.at[0, 'Paikka_Id']

    for j, subwatershed in subwatersheds.iterrows():

        subwatershed = geopd.GeoDataFrame(
                dict(zip(list(subwatershed.index), list(subwatershed.values))),
                crs=subwatersheds.crs, geometry='geometry', index=[0])

        minx = subwatershed.bounds.at[0, 'minx']
        miny = subwatershed.bounds.at[0, 'miny']
        maxx = subwatershed.bounds.at[0, 'maxx']
        maxy = subwatershed.bounds.at[0, 'maxy']

        with rasterio.open(land_cover_path) as src:
            profile = src.profile
            values = src.read(
                1, window=from_bounds(minx, miny, maxx, maxy, src.transform),
                boundless=True, fill_value=profile['nodata'])
        
        profile['transform'] = rasterio.transform.from_bounds(minx, miny, maxx, maxy, values.shape[1], values.shape[0])
        profile['width'] = values.shape[1]
        profile['height'] = values.shape[0]
        
        area_mask = rasterize(
                subwatershed['geometry'], (profile['height'], profile['width']),
                dtype=profile['dtype'], transform=profile['transform'], all_touched=True)
        
        area = area_mask.sum()
        
        clipped_values = np.where(area_mask == 1, values, 0)
        for key in lc_classes:
            class_name = lc_classes[key]
            class_values = np.where(clipped_values == key, 1, 0)
        
            class_area = class_values.sum()
    
            # Some subwaterhseds cause minor problems 
            warnings.filterwarnings("error")
            
            try:
                class_portion = class_area / area
    
            except:
                class_portion = 0
                print(f"{class_name}_perc was set to zero for watershed {place_id}, subwatershed {j} because {class_area} or {area} were invalid features")
            warnings.filterwarnings("default")
            
            # Getting percentages
            subwatersheds.at[j, f"{class_name}_perc_{year}"] = class_portion * 100

    return subwatersheds

In [8]:
# Main processing loop
dst_path = '/path/to/filename.gpkg'
pbar = tqdm(watersheds.iterrows(), total=len(watersheds))

for i, watershed in pbar:
    pbar.set_description(f"Processing catchment {i}")
    # Changing to GeoDataFrame
    watershed = geopd.GeoDataFrame(
                    dict(zip(list(watershed.index), list(watershed.values))),
                    crs=watersheds.crs, geometry='geometry', index=[0])
    
    minx = watershed.bounds.at[0, 'minx']
    miny = watershed.bounds.at[0, 'miny']
    maxx = watershed.bounds.at[0, 'maxx']
    maxy = watershed.bounds.at[0, 'maxy']
    
    """
    #Land cover
    """
    # Optional progress bar
    #pbar.set_description(f"Doing land cover for catchment {i}")
    year_values = []
    counter = 0
    for lc_year in land_cover_paths:
        land_cover_path = land_cover_paths[lc_year]
        watershed = add_lc_to_watersheds(watershed, land_cover_path, lc_classes, lc_year)
        # the end result shouldnt have multiple columns with same name and values
        #if counter > 0:
            #watershed_lc.drop(['geometry', 'area', 'Paikka_Id'], axis=1)
        #year_values.append(watershed_lc)
        counter += 1
    #watershed = pd.concat(year_values, axis=1)
    
    watersheds.loc[i, lc_columns] = watershed.loc[0, lc_columns]

    """
    #Soil
    """
    #pbar.set_description(f"Doing soil class for catchment {i}")
    
    soil = geopd.read_file(soil_path, bbox=(minx, miny, maxx, maxy))
    soil = soil.clip(watershed, keep_geom_type=True)

    # There is soil data
    if len(soil) > 0:
        for soil_class in soil_classes:
            # Classifying into given classes
            soil.loc[soil["PINTAMAALAJI_KOODI"].isin(soil_classes[soil_class]), "soil_class"] = soil_class
            
        soil = soil.dissolve(by="soil_class", as_index=False)
        watershed_area = watershed.area.sum()
            
        for soil_class in soil_classes:
            soil_class_area = soil[soil["soil_class"] == soil_class].area.sum()
            # multiplying by 100 to get percentage
            watersheds.at[i, f"{soil_class}_perc"] = round((soil_class_area / watershed_area) * 100, 4)

    # Handling nodata situation
    else:
        for soil_class in soil_classes:
            watersheds.at[i, f"{soil_class}_perc"] = 0
    """
    #Soil depth
    """
    #pbar.set_description(f"Doing soil depth for catchment {i}")

    # Opening window of the data from the area of the watershed    
    with rasterio.open(soil_depth_path) as src:
        profile = src.profile
        values = src.read(
            1, window=from_bounds(minx, miny, maxx, maxy, src.transform),
            boundless=True, fill_value=profile['nodata'])
    
    profile['transform'] = rasterio.transform.from_bounds(minx, miny, maxx, maxy, values.shape[1], values.shape[0])
    profile['width'] = values.shape[1]
    profile['height'] = values.shape[0]

    # The typical, non empty case
    if profile['width'] and profile['height'] > 0:
    
        # Nodata and waters are excluded so that they don't interfere
        masked_values = ma.masked_values(values, profile['nodata'])
        # Waters are represented as -99 and are removed
        masked_values = ma.masked_values(masked_values, -99)
        
        area_mask = rasterize(
                    watershed['geometry'], (profile['height'], profile['width']),
                    dtype=profile['dtype'], transform=profile['transform'], all_touched=True)
    
        # When calculating the average, unwanted regions must be masked
        clipped_values = ma.masked_where(area_mask == 0, masked_values)
        
        # If the area is completely covered by lakes, the value is set to 0
        warnings.filterwarnings("error") 
        try:
            average_depth = round(clipped_values.sum() / clipped_values.count(), 4)
    
        except:
            average_depth = 0
            print(f"soil depth was set to zero for watershed {i}, because there were no non-masked values")
        warnings.filterwarnings("default") 
    
        watersheds.at[i, f"soil_depth"] = average_depth
    # Handling the empty exception
    else:
        print(f"soil depth was set to zero for watershed {i}, because there were no non-masked values")
        watersheds.at[i, f"soil_depth"] = 0

    """
    #Slope
    """
    #pbar.set_description(f"Doing slope for catchment {i}")

    with rasterio.open(slope_path) as src:
            profile = src.profile
            values = src.read(
                1, window=from_bounds(minx, miny, maxx, maxy, src.transform),
                boundless=True, fill_value=profile['nodata'])
        
    profile['transform'] = rasterio.transform.from_bounds(minx, miny, maxx, maxy, values.shape[1], values.shape[0])
    profile['width'] = values.shape[1]
    profile['height'] = values.shape[0]

    # Nodata and waters are excluded so that they don't interfere
    masked_values = ma.masked_values(values, profile['nodata'])

    
    area_mask = rasterize(
            watershed['geometry'], (profile['height'], profile['width']),
            dtype=profile['dtype'], transform=profile['transform'], all_touched=True)

    # When calculating the average, unwanted regions must be masked
    clipped_values = ma.masked_where(area_mask == 0, masked_values)
    
    # If the area is completely covered by lakes, the value is set to 0
    warnings.filterwarnings("error") 
    try:
        average = round(clipped_values.sum() / clipped_values.count(), 4)

    except:
        average = 0
        print(f"slope was set to zero for watershed {i}, because there were no values")

    warnings.filterwarnings("default") 
    watersheds.at[i, f"slope"] = average

    """
    #Elevation
    """
    
    # Opening window of the data from the area of the watershed    
    with rasterio.open(dem_path) as src:
        profile = src.profile
        values = src.read(
            1, window=from_bounds(minx, miny, maxx, maxy, src.transform),
            boundless=True, fill_value=profile['nodata'])
    
    profile['transform'] = rasterio.transform.from_bounds(minx, miny, maxx, maxy, values.shape[1], values.shape[0])
    profile['width'] = values.shape[1]
    profile['height'] = values.shape[0]

    # Nodata and waters are excluded so that they don't interfere
    masked_values = ma.masked_values(values, profile['nodata'])

    
    area_mask = rasterize(
            watershed['geometry'], (profile['height'], profile['width']),
            dtype=profile['dtype'], transform=profile['transform'], all_touched=True)

    # When calculating the average, unwanted regions must be masked
    clipped_values = ma.masked_where(area_mask == 0, masked_values)
    nan_clip_values = clipped_values.filled(np.nan)

    
    #Calculating elevation descriptors

    # Mean
    watersheds.at[i, 'elev_mean'] = np.nanmean(nan_clip_values)
    # Minimum
    watersheds.at[i, 'elev_min'] = clipped_values.min()
    
    # 10th percentile
    watersheds.at[i, 'elev_10'] = np.nanpercentile(nan_clip_values, 10)
    # median
    watersheds.at[i, 'elev_50'] = np.nanpercentile(nan_clip_values, 50)
    # 90 th percentile
    watersheds.at[i, 'elev_90'] = np.nanpercentile(nan_clip_values, 90)

    #Maximum 
    watersheds.at[i, 'elev_max'] = clipped_values.max()
    
    
watersheds['elev_range'] =  watersheds['elev_max'] - watersheds['elev_min']
watersheds.to_file(dst_path, layer="v1", driver="GPKG")

  0%|          | 0/320 [00:00<?, ?it/s]

In [11]:
watersheds.to_file(src_path, layer="attributes_v1", driver="GPKG")

In [ ]:
watersheds